## Reproduce datasets for Scheme B

Paper: "Statistical supervised meta-ensemble algorithm for data linkage"

Kha Vo, Jitendra Jonnagaddala, Siaw-Teng Liaw

February 2019

Jounal of Biomedical Informatics

In [1]:
inputfile = 'ePBRN_F_original' # change between 'ePBRN_D_original' or 'ePBRN_F_original'
outputfile = 'ePBRN_F_dup' # change between 'ePBRN_D_dup' or 'ePBRN_F_dup'

In [3]:
import pandas as pd
import numpy as np
import random
from numpy.random import choice
import sys

In [19]:
# Input the percentage of  [2, 3, 4] shared records in one linkage:
count_shared = [1.68+21.0659, 1.9986 + 0.0471, 0.05]

# Assigning the weights for each type of error:

abr = 1 # abbreviation on surname: Michael -> M
jwd1 = 1 # join with dash: John Peter -> John-Peter, join surname and given name into surname
jwd2 = 1 # join with dash: John Peter -> John-Peter, join surname and given name into given name
jwb1 = 1 # join with blank: 
jwb2 = 1 # join with blank: 
drf = 1 # drop all tokens in any field
dlc1 = 1 # drop last character in surname: Peter -> Pete
dlc2 = 1 # drop last character in given name
swn = 1 # swap surname and given name: John Peter -> Peter John
swd = 1 # swap day and month fields: 12/04 -> 04/12
rsd = 1 # reset day and month: 12/04/1991 -> 01/01/1991
chy = 1 # change year of birth by a margin of (+/-)5 
chz = 1 # change any number of digit from zip code
mar = 1 # change the whole token of surname: Mary Ward -> Mary Winston
twi = 1 # duplicate all fields except given name: Micheal Williams -> Leo Williams
add = 1 # change the whole 3 fields of address by randomly replacing each field by any other row

all_error_types = ['abr','jwd1','jwd2','jwb1','jwb2' ,'drf','dlc1','dlc2','swn',
                   'rsd','chy','chz','mar','twi','add']
all_error_weights = [abr, jwd1, jwd2, jwb1, jwb2, drf, dlc1, dlc2, swn, rsd, chy, chz, mar, twi, add]
all_error_weights = all_error_weights/sum(np.asarray(all_error_weights))

# COMMENT(alecmori): If all errors {jwd1, jwd2, jwb1, jwb2} are enabled,
# rc['surname'] == rc[surname] + "-" + rc[given_name] + "-  " (it keeps augmenting the last name).
# This has major negative repercussions in later downstream branches.
# We should clean this up for experiments.
def process_record(rc):
    no_error = np.random.poisson(1, 1)
    errortypes = choice(all_error_types, no_error, p=all_error_weights)
    for errortype in errortypes:
        if errortype == 'abr':
            if len(rc["surname"])>0:
                #rc["surname"] = rc["surname"][0]
                rc.at["surname"] = rc["surname"][0]
        if errortype == 'jwd1':
            rc.at["surname"] = rc["surname"] + '-' + rc["given_name"]
            rc.at["given_name"] = ''
        if errortype == 'jwd2':
            rc.at["given_name"] = rc["surname"] + '-' + rc["given_name"]
            rc.at["surname"] = ''
        if errortype == 'jwb1':
            rc.at["surname"] = rc["surname"] + ' ' + rc["given_name"]
            rc.at["given_name"] = ''
        if errortype == 'jwb2':
            rc.at["given_name"] = rc["surname"] + ' ' + rc["given_name"]
            rc.at["surname"] = ''
        if errortype == 'drf':    
            selected_field = random.choice(all_fields)
            rc.at[selected_field] = ''
        if errortype == 'dlc1':
            if len(rc["surname"])>0:
                rc.at["surname"] = rc['surname'][0:-1]
        if errortype == 'dlc2':
            if len(rc["given_name"])>0:
                rc.at["given_name"] = rc['given_name'][0:-1]
        if errortype == 'swn':
            temp = rc['given_name']
            rc.at["given_name"] = rc['surname']
            rc.at["surname"] = temp
        if errortype == 'swd': 
            temp = rc['day']
            rc.at['day'] = rc['month']
            rc.at['month'] = temp
        if errortype == 'rsd':
            rc.at['day'] = '01'
            rc.at['month'] = '01'
        if errortype == 'chy': 
            if rc.at['year'] != 'NaT' and rc['year'] != '':
                margin = random.choice(range(-5,5))
                rc.at['year'] = str( int(rc['year']) + margin)
        if errortype == 'chz':
            if len(str(rc['postcode']))== 4:
                selected_digit = random.choice(range(4))
                code = list(str(rc['postcode']))
                code[selected_digit] = str( random.choice(range(9)))
                rc.at['postcode'] = int(''.join(code))
        if errortype == 'mar':
            rc.at["surname"] = df.iloc[random.choice(range(len(df)))]['surname']
        if errortype == 'twi':
            rc.at["given_name"] = df.iloc[random.choice(range(len(df)))]['given_name']
        if errortype == 'add':
            rc.at['address_1'] = df.iloc[random.choice(range(len(df)))]['address_1']
            rc.at['address_2'] = df.iloc[random.choice(range(len(df)))]['address_2']
            rc.at['street_number'] = random.choice(range(500))
    return rc

In [12]:
# Preprocess
df = pd.read_csv(
    f"ePBRN_Datasets/{inputfile}.csv", 
    parse_dates=["date_of_birth"])

df["rec_id"] = range(len(df))
df['rec_id'] = df['rec_id'].astype(str)
df['day'] = df['date_of_birth'].dt.strftime('%d')
df['month'] = df['date_of_birth'].dt.strftime('%m')
# Comment(alecmori): Added fillna here due to a missing data point.
df['year'] = df['date_of_birth'].dt.strftime('%Y').fillna(0)
df['postcode'] =   df['postcode'].fillna('0000')
df['postcode'] = df['postcode'].astype(int)
df['street_number'] =   df['street_number'].fillna('0')
df['street_number'] = df['street_number'].astype(int)
df = df.drop(["age", "phone_number", "soc_sec_id", "blocking_number", "date_of_birth"], axis=1)
for col in ["surname", "given_name", "address_1", "address_2", "day", "month"]:
    df[col] = df[col].fillna('')
    df[col] = df[col].astype(str)
df["match_id"] = range(len(df))
all_fields = df.columns.values.tolist()
all_fields.remove('rec_id')

print("Original dataset length:",len(df))
df.head()


Original dataset length: 11100


,rec_id,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,day,month,year,match_id
0,0,jenna,kilpin,179,mcfarlan place,,hillarys,2768,vic,26,02,1950,0
1,1,bianca,randazzo,37,lindrum crescent,sunshine,forster,2281,wa,07,04,1988,1
2,2,james,borlase,75,,rocklea,casula,2460,qld,03,09,1913,2
3,3,nicholas,beeton,20,mugga way,,hawthorn,2480,vic,22,06,1999,3
4,4,megan,footner,4,jewell close,,taylors lakes,3129,tas,22,09,1912,4


In [10]:
# Generate random indices for linkages
leng = len(df)
print("Process", inputfile, ", total records:", leng, "...")
# Comment(alecmori): Why are we doing this hacky thing to count how many of each link there are
# instead of actually counting them.
no_double_linked = int(leng*count_shared[0]/100)
no_triple_linked = int(leng*count_shared[1]/100)
no_quad_linked = int(leng*count_shared[2]/100)
list_double_linked = random.sample(range(leng),k=no_double_linked)
remain = [item for item in range(leng) if item not in list_double_linked]
list_triple_linked = random.sample(remain,k=no_triple_linked)
remain = [item for item in remain if item not in list_triple_linked]
list_quad_linked = random.sample(remain,k=no_quad_linked)
print("Double links:", no_double_linked,". Triple links:",no_triple_linked,". Quad links:",no_quad_linked)
print("Total records after generated:", leng + no_double_linked + no_triple_linked*2 + no_quad_linked*3)
print("Matched pairs:", no_double_linked + no_triple_linked*3 + no_quad_linked*6)

Process ePBRN_F_original , total records: 11100 ...
Double links: 2524 . Triple links: 227 . Quad links: 5
Total records after generated: 14093
Matched pairs: 3235


In [22]:
# Main steps
df1 = df
j = 0
for list_linked in [list_double_linked, list_triple_linked, list_quad_linked]:
    j = j + 1
    for i in list_linked:
        for k in range(j):
            record_to_process = df.iloc[i]
            processed_rc = process_record(record_to_process)
            rec_id = processed_rc["rec_id"]
            processed_rc.at["rec_id"] = f"{rec_id}-dup-{k}"
            df1 = df1.append(processed_rc)

# Save to disk    
outputpath = f"ePBRN_Datasets/{outputfile}.csv"
df1.to_csv(outputpath, index=False)
print("Saved to", outputpath)

<ipython-input-19-f7908406ed85>:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rc.at["given_name"] = rc['surname']
<ipython-input-19-f7908406ed85>:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rc.at["surname"] = temp
<ipython-input-19-f7908406ed85>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rc.at["given_name"] = rc["surname"] + ' ' + rc["given_name"]
<ipython-input-19-f7908406ed85>:51: SettingWithCopyWarning: 
A value is 

<ipython-input-22-6f0b4c3ea0bb>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append(processed_rc)
<ipython-input-22-6f0b4c3ea0bb>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_rc.at["rec_id"] = f"{rec_id}-dup-{k}"
<ipython-input-22-6f0b4c3ea0bb>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append(processed_rc)
<ipython-input-22-6f0b4c3ea0bb>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_

<ipython-input-19-f7908406ed85>:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rc.at["surname"] = temp
<ipython-input-22-6f0b4c3ea0bb>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_rc.at["rec_id"] = f"{rec_id}-dup-{k}"
<ipython-input-22-6f0b4c3ea0bb>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append(processed_rc)
<ipython-input-19-f7908406ed85>:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.

<ipython-input-19-f7908406ed85>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rc.at["given_name"] = rc["surname"] + ' ' + rc["given_name"]
<ipython-input-19-f7908406ed85>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rc.at["surname"] = ''
<ipython-input-19-f7908406ed85>:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rc.at["given_name"] = df.iloc[random.choice(range(len(df)))]['given_name']
<ipython-input-22-6f0b4c3ea0bb>:11:

<ipython-input-22-6f0b4c3ea0bb>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_rc.at["rec_id"] = f"{rec_id}-dup-{k}"
<ipython-input-22-6f0b4c3ea0bb>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append(processed_rc)
<ipython-input-19-f7908406ed85>:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rc.at['year'] = str( int(rc['year']) + margin)
<ipython-input-19-f7908406ed85>:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation:

...

<ipython-input-22-6f0b4c3ea0bb>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append(processed_rc)
<ipython-input-22-6f0b4c3ea0bb>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_rc.at["rec_id"] = f"{rec_id}-dup-{k}"
<ipython-input-22-6f0b4c3ea0bb>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append(processed_rc)
<ipython-input-19-f7908406ed85>:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rc.at['yea

<ipython-input-22-6f0b4c3ea0bb>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_rc.at["rec_id"] = f"{rec_id}-dup-{k}"
<ipython-input-22-6f0b4c3ea0bb>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append(processed_rc)
<ipython-input-22-6f0b4c3ea0bb>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_rc.at["rec_id"] = f"{rec_id}-dup-{k}"
<ipython-input-22-6f0b4c3ea0bb>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inst

<ipython-input-19-f7908406ed85>:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rc.at['street_number'] = random.choice(range(500))
<ipython-input-22-6f0b4c3ea0bb>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_rc.at["rec_id"] = f"{rec_id}-dup-{k}"
<ipython-input-22-6f0b4c3ea0bb>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append(processed_rc)
<ipython-input-19-f7908406ed85>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentat

Saved to ePBRN_Datasets/ePBRN_F_dup.csv


<ipython-input-22-6f0b4c3ea0bb>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_rc.at["rec_id"] = f"{rec_id}-dup-{k}"
<ipython-input-22-6f0b4c3ea0bb>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append(processed_rc)


In [24]:
df1.tail(25)

,rec_id,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,day,month,year,match_id
3923,3923-dup-0,medhurst stephanie,,2,livingston avenue,,jimboomba,2066,qld,01,03,1998,3923
3923,3923-dup-1,stephani,medhurst,2,livingston avenue,,jimboomba,2066,qld,01,03,2002,3923
5523,5523-dup-0,nathan,smith,19,magennis place,,northgate,2087,vic,15,08,1968,5523
5523,5523-dup-1,nathan,smith,19,magennis place,,northgate,2087,vic,15,08,1968,5523
9588,9588-dup-0,george,adams,5,springvale drive,,wynyard,3429,nsw,06,08,1964,9588
9588,9588-dup-1,george,adams,5,springvale drive,,wynyard,3429,nsw,06,08,1964,9588
9285,9285-dup-0,connor,scott,61,blancoa place,,thornton,2203,qld,,,0,9285
9285,9285-dup-1,,rzemieniuk-connor,61,blancoa place,,thornton,2203,qld,,,0,9285
3075,3075-dup-0,andrew,knight,6,verbrugghen street,,nedlands,3021,qld,06,10,1912,3075
3075,3075-dup-1,andrew,knight,6,verbrugghen street,,nedlands,3021,qld,06,10,1912,3075


In [20]:
len(df)

11100